# Чекпоинт №3

# Отчет по vision-модулю для Minecraft-агента

## Постановка задачи и данные
Разработан модуль компьютерного зрения для многолейбловой классификации 119 бинарных признаков среды Minecraft по скриншотам разрешения 360×640 пикселей. Целью является предоставление агенту структурированного вектора состояния мира, необходимого для навигации и принятия решений. Признаковое пространство разделено на критические группы (биом, время суток, GUI, видимость объектов, враги) и второстепенные (ресурсы инвентаря, инструменты). Обучение и валидация проводились на синтетическом датасете из 10 000 примеров, сгенерированном через FakeBackend, с разбиением 80/20.

![alt text](images/image.png)

## Метрики оценки
В качестве ключевой метрики эффективности выбрана `macro_F1_critical`. Она представляет собой среднее гармоническое точности и полноты исключительно по критическим тегам, определяющим выживание и базовую навигацию агента. Общее качество классификации отслеживается через вспомогательные метрики `macro_F1_all`, `micro_F1_all` и `hamming_loss`. Приоритет отдан безошибочному определению опасностей и контекста, а не точному подсчету ресурсов в инвентаре.

## Архитектура и эксперименты
Для извлечения признаков использован энкодер ResNet18, предобученный на ImageNet. Сравнивались три подхода: KNN (k=5) и логистическая регрессия на фиксированных 512-мерных векторах признаков, а также полностью дообученная сеть ResNet с модифицированным выходным слоем. Валидация проводилась на едином отложенном наборе данных.

![alt text](images/image-1.png)

## Анализ результатов
Логистическая регрессия и Fine-tuned ResNet показали практически идеальный результат по ключевой метрике `macro_F1_critical` (0.9997), превзойдя KNN (0.9706). Однако на полном наборе из 119 тегов лидером оказался KNN (`macro_F1_all` 0.73) за счет лучшей работы с редкими классами ресурсов, где ResNet показал значительное падение производительности (0.49). `Hamming_loss` минимален у KNN, что подтверждает его статистическую точность на всем распределении, но для целевой задачи агента приоритетна надежность именно критических групп.

## Детализация по классам
Анализ по меткам подтверждает гипотезу о достаточности данных для критических задач. Все теги категорий биома, времени суток, GUI и видимости врагов имеют F1 равный или близкий к 1.0. Ошибки классификации сосредоточены исключительно в области редких меток ресурсов с поддержкой менее 50 примеров на датасет, что объясняется дисбалансом классов при синтетической генерации и не влияет на базовую функциональность агента.

![alt text](images/image-2.png)
![alt text](images/image-3.png)
![alt text](images/image-4.png)

## Гиперпараметры
Перебор гиперпараметров выявил низкую чувствительность моделей к настройкам. Для KNN оптимальным оказалось количество соседей равное 7 при пороге 0.5. Логистическая регрессия показала стабильные результаты при C=1.0 и пороге 0.3. Это свидетельствует о линейной разделимости признакового пространства, сформированного ResNet18, для выбранных категорий.

![alt text](images/image-5.png)
![alt text](images/image-6.png)

## Итоговое заключение
Vision-модуль на базе архитектуры ResNet успешно решает задачу формирования вектора состояния среды. Несмотря на преимущество KNN в общих метриках, для интеграции в агента рекомендован подход на базе ResNet (end-to-end) или связки Frozen ResNet + LogReg. Оба варианта обеспечивают безошибочное (F1 ~1.0) детектирование критических условий, необходимых для работы планировщика. Модуль признан готовым к внедрению в контур управления агента.


# Отчет по системе планирования действий

## **1. Изначальная постановка задачи**

Первоначальная цель данного модуля состояла в создании системы, которая

1. Принимает пользовательский запрос (например: «мне нужна алмазная кирка» или «сделай кварцевые кирпичи»).
2. Определяет, какой предмет является целью.
3. Строит детальный пошаговый план, как эту цель достичь, то есть

   - что добыть,
   - что переплавить,
   - что скрафтить,
   - какие инструменты нужны и в каком порядке.

Изначально предполагалось, что всю эту работу будет делать дообученная GPT-2, и планировалось:

1. собрать датасет крафтов и инструкций,
2. дообучить GPT-2,
3. использовать LLM как единственный компонент, выполняющий и классификацию, и планирование.

---

## **2. Первая реализация и возникшие проблемы**

Изначально план был следующий:

- собрать большой датасет (пары _goal → steps_);
- обучить GPT-2 воспроизводить пошаговые планы;
- заставить модель опираться на рецепты и структуру мира.

### **Входные данные**
**Использовалась следующая модель:**
- Модель: **GPT-2 (base, ~124M параметров)**  
- Фреймворк: `transformers` (HuggingFace)
- Задача обучения: **causal language modeling**

Получилось с помощью разных ML-инструментов и ручной работы вывести датасет на 5 тысяч запросов, учитывая мультиязычность, вида:
```json
{"instruction": "Gather iron ore from a cave.", "steps": ["find a cave", "go to the cave entrance", "enter the cave", "find iron ore", "mine the iron ore", "go back to the cave entrance"]}
```

**После рефакторинга под обучение он приобрел вид:**
```
Instruction: Craft a diamond pickaxe.
Steps:
1. find a tree
2. chop the tree
3. collect oak logs
...
<END>
```

**Каждый пример содержал**
- `Instruction` — текстовая формулировка задачи;
- `Steps` — линейный список действий, которые **должны быть выполнимы STEVE-1**;
- служебный токен `<END>` для принудительного окончания генерации.

**Датасет включал**
- крафт инструментов разных материалов;
- добычу ресурсов;
- базовые действия выживания;
- постройка разных классических построек;

### Процесс обучения
- Обучение проводилось 3–5 эпох
- Loss стабильно снижался (примерно с 2.6557 до 0.2332)
- Использовался greedy decoding или ограниченный sampling
- Специальных метрик качества (BLEU / ROUGE) не вводилось, так как результат оценивался **качественно** - по корректности плана

При не самом огромном количестве данных получилось довести модель до loss 0.2332
![loss](images/gpt-2-loss.png)

Но даже при таком значении коэффициента ошибок GPT-2 абсолютно не понимал статус выполнения задачи и саму цель запроса. То есть, к примеру, при запросе "craft a diamond pickaxe" результат был примерно такой 

```
1) find a flat area
2) collect oak logs
3) craft oak planks
4) build the house floor
5) build the walls
6) build the roof
7) place a door
8) place a door
9) place a door
10) place a door
11) place a door
12) place a door
13) place a door
14) place a door
15) place a door
16) place a door
17) place a door
18) place a door
19) place a door
20) place a door
21) place a door
22) place a door
23) place a door
24) place a door
25) place a door
```

Либо с другими параметрами обучения при запросе "Enter the nether"

```
1) find a tree
2) chop the tree
3) collect oak logs
4) craft oak planks
5) craft sticks
6) craft a wooden pickaxe
7) find stone
8) mine stone
9) collect cobblestone
10) craft a stone pickaxe
11) find a cave
12) enter the cave
13) find iron ore
14) mine iron ore
15) collect iron ore
16) craft a furnace
17) smelt iron ore
18) collect iron ingots
19) craft an iron pickaxe
20) find gold ore
21) mine gold ore
22) collect gold ore
23) craft a furnace
24) smelt gold ore
25) collect gold ingots
26) craft a furnace
27) smelt gold ingots
28) collect gold ingots
29) craft a furnace
30) smelt gold ore
31) collect gold ingots
```

Хотя в этот раз у модели получилось успешно прийти к крафту алмазной кирки, хоть на этом остановки и не было

```
1) find a tree
2) chop the tree
3) collect oak logs
4) craft oak planks
5) craft sticks
6) craft a wooden pickaxe
7) find stone
8) mine stone
9) collect cobblestone
10) craft a stone pickaxe
11) find a cave
12) enter the cave
13) find iron ore
14) mine iron ore
15) collect iron ore
16) craft a furnace
17) smelt iron ore
18) collect iron ingots
19) craft an iron pickaxe
20) find diamond ore
21) mine diamond ore
22) collect diamonds
23) craft a diamond pickaxe
24) craft a diamond shovel
25) find diamond ore
26) mine diamond ore
27) collect diamonds
28) craft a diamond pickaxe
29) craft a diamond shovel
30) find gold ore
31) mine gold ore
32) collect
```

Исходя из всех тестов мы вывели фундаментальные проблемы.

### **Проблема 1**

GPT-2 периодически добавляла:

- лишние предметы;
- несуществующие шаги;
- неправильное количество ресурсов;
- неправильный порядок действий;
- дублирование одного и того же шага множество раз;

### **Проблема 2**

Модель могла:

- пытаться добыть алмаз каменной киркой;
- забывать про изначальные ресурсы;
- пытаться крафтить золотую кирку, потому что, судя по всему, ей добывать удобнее;
- пытаться засадить огород или убить моба при запросе крафта печки;

### **Проблема 3**

STEVE-1 принимает только фиксированные базовые действия, а LLM не гарантировала

- что действия будут из whitelist;
- что они будут детерминированы;
- что не появится «построить дом», «создать портал» и т.п;
- что действия будут окончательными;

Насчет последнего поясню, LLM (по крайней мере GPT-2 без генеративного трансформера, преанализа и прочего) не понимает такого "ивента", как "достижение цели" и продолжает генерацию, даже если все уже сделано, либо наоборот обрывался раньше, потому что после данного шага чаще всего идет конец.

### **К чему мы пришли после данных выводов**

> Использовать LLM как полноценный планировщик - нестабильный, недетерминированный и трудно контролируемый компонент.

---

## **3. Смена архитектуры**

Было принято решение полностью перестроить систему:

1. Множество Minecraft-данных переносятся в граф зависимостей (CRAFT / MINE / SMELT).
2. Детерминированный планировщик разбирает цель на шаги.
3. Классификатор (embedding-based) определяет, что именно хочет пользователь.

---

## **4. Загрузка и нормализация данных Minecraft**

В качестве источника данных использован проект:

> **[PrismarineJS / minecraft-data](https://github.com/PrismarineJS/minecraft-data/tree/master)**

Загружены:

| Файл                                 | Назначение                                                                |
| ------------------------------------ | ------------------------------------------------------------------------- |
| `recipes.json`                       | рецепты крафта                                                            |
| `items.json`                         | полная база предметов                                                     |
| `blocks.json`                        | информация о блоках, дропа, прочности                                     |
| `materials.json`, `instruments.json` | второстепенные таблицы                                                    |
| `language.json`                      | локализация (к сожалению, пока не использовалась и не оправдала ожидания) |

На их основе построена собственная единая база действий (`actions_db.json`).

---

## **5. Генерация базы действий**

Созданы **три типа действий**:

### **1. MINE**

- берётся каждый diggable-блок,
- дропы извлекаются из `blocks.json`,
- корректно учитывается Minecraft-логика, связанная с рудами, а точнее

```
iron_ore → raw_iron
gold_ore → raw_gold
diamond_ore → diamond
```

Структура таких данных следующая:

```json
{
	"id": "MINE:cobbled_deepslate_slab",
	"kind": "MINE",
	"result": "cobbled_deepslate_slab",
	"requires_tool": "wooden_pickaxe"
}
```

### **2. SMELT**

Добавлены правила Minecraft:

- `raw_* → *_ingot`
- `sand → glass`
- `cobblestone → stone`
- `clay_ball → brick`

Структура таких данных следующая:

```json
  {
    "id": "SMELT:raw_iron->iron_ingot",
    "kind": "SMELT",
    "input": "raw_iron",
    "result": "iron_ingot"
  },
```

### **3. CRAFT**

- автоматически парсятся все рецепты,
- корректно подсчитываются ингредиенты,
- добавляются требования (`requires`):

  - `crafting_table`
  - `furnace`.

**Циклы (например, diamond → diamond_block → diamond) были автоматически обнаружены и исключены.**

Структура таких данных следующая:

```json
  {
    "id": "CRAFT:black_bed:6",
    "kind": "CRAFT",
    "result": "black_bed",
    "inputs": [
      {
        "item": "black_wool",
        "count": 3
      },
      {
        "any_of": [
          "oak_planks",
          "spruce_planks",
          "birch_planks",
          "jungle_planks",
          "acacia_planks",
          "cherry_planks",
          "dark_oak_planks",
          "pale_oak_planks",
          "mangrove_planks",
          "bamboo_planks",
          "crimson_planks",
          "warped_planks"
        ],
        "count": 3
      }
    ],
    "count": 1
  },
```

---

## **6. Классификация предметов**

Каждый предмет получил класс:

- **GOAL** — то, что пользователь реально хочет
  (мотыги, кирки, броня, сундук, верстак)
- **RESOURCE** — то, что добывается или является базовым ресурсом
  (бревно, руда, доска)
- **INTERMEDIATE** — ингредиенты
  (слитки, палки, камень)

---

## **7. Построение графа зависимостей**

Построен граф вида:

```
item → [actions которые могут его произвести]
action → [inputs]
```

Учтено:

- отсутствие циклов,
- поддержка действий ИЛИ («палки можно из любых досок»),
- автоматическое объединение эквивалентных ресурсов (plank → stick),
- приведение всех видов дерева к единой операции:

  ```
  ANY_LOG → ANY_PLANK → stick
  ```

---

## **8. Детерминированный планировщик**

Создан собственный рекурсивный planner, который:

- берёт желаемый предмет (например, diamond_pickaxe),
- вычисляет минимальный путь:

```
CHOP tree
CRAFT planks
CRAFT stick
CRAFT wooden_pickaxe
MINE cobblestone
CRAFT stone_pickaxe
MINE raw_iron
SMELT raw_iron → iron_ingot
CRAFT iron_pickaxe
MINE diamond
CRAFT diamond_pickaxe
```

Для объективной оценки были введены структурные метрики:
| Метрика            | Назначение                      |
| ------------------ | ------------------------------- |
| `steps`            | длина плана                     |
| `max_depth`        | глубина зависимостей            |
| `CRAFT/MINE/SMELT` | распределение действий          |
| `determinism`      | доля шагов без выбора           |
| `unique_items`     | количество уникальных ресурсов  |


В рамках эксперимента был задан фиксированный набор тестовых целей:
- wooden_pickaxe;
- stone_pickaxe;
- iron_pickaxe;
- diamond_pickaxe;
- crafting_table;
- furnace;
- blast_furnace;
- anvil;
- chest;
- barrel;
- hopper;
- glass;
- stone_bricks;
- quartz_block;
- quartz_bricks;
- iron_ingot;
- gold_ingot;
- copper_block;

Результаты оказались следующими
![steps](images/eval_steps_per_goal.png)

![depth](images/eval_depth_per_goal.png)

```
=== GOAL: wooden_pickaxe ===
1. MINE oak_log
2. CRAFT oak_planks
3. CRAFT stick
4. CRAFT wooden_pickaxe
METRICS: {'goal': 'wooden_pickaxe', 'steps': 4, 'max_depth': 2, 'craft': 3, 'mine': 1, 'smelt': 0, 'determinism': 0.5, 'unique_items': 4}

=== GOAL: stone_pickaxe ===
1. MINE oak_log
2. CRAFT oak_planks
3. CRAFT stick
4. CRAFT wooden_pickaxe
5. MINE stone
6. CRAFT stone_pickaxe
METRICS: {'goal': 'stone_pickaxe', 'steps': 6, 'max_depth': 4, 'craft': 4, 'mine': 2, 'smelt': 0, 'determinism': 0.6666666666666666, 'unique_items': 6}

=== GOAL: iron_pickaxe ===
1. MINE oak_log
2. CRAFT oak_planks
3. CRAFT stick
4. CRAFT wooden_pickaxe
5. MINE stone
6. CRAFT stone_pickaxe
7. MINE raw_iron
8. SMELT raw_iron -> iron_ingot
9. CRAFT iron_pickaxe
METRICS: {'goal': 'iron_pickaxe', 'steps': 9, 'max_depth': 7, 'craft': 5, 'mine': 3, 'smelt': 1, 'determinism': 0.7777777777777778, 'unique_items': 9}

=== GOAL: diamond_pickaxe ===
1. MINE oak_log
2. CRAFT oak_planks
3. CRAFT stick
4. CRAFT wooden_pickaxe
5. MINE stone
6. CRAFT stone_pickaxe
7. MINE raw_iron
8. SMELT raw_iron -> iron_ingot
9. CRAFT iron_pickaxe
10. MINE diamond
11. CRAFT diamond_pickaxe
METRICS: {'goal': 'diamond_pickaxe', 'steps': 11, 'max_depth': 9, 'craft': 6, 'mine': 4, 'smelt': 1, 'determinism': 0.8181818181818182, 'unique_items': 11}

=== GOAL: crafting_table ===
1. MINE oak_log
2. CRAFT oak_planks
3. CRAFT crafting_table
METRICS: {'goal': 'crafting_table', 'steps': 3, 'max_depth': 2, 'craft': 2, 'mine': 1, 'smelt': 0, 'determinism': 0.6666666666666666, 'unique_items': 3}

=== GOAL: furnace ===
1. MINE oak_log
2. CRAFT oak_planks
3. CRAFT stick
4. CRAFT wooden_pickaxe
5. MINE stone
6. CRAFT furnace
METRICS: {'goal': 'furnace', 'steps': 6, 'max_depth': 4, 'craft': 4, 'mine': 2, 'smelt': 0, 'determinism': 0.6666666666666666, 'unique_items': 6}

=== GOAL: blast_furnace ===
1. MINE oak_log
2. CRAFT oak_planks
3. CRAFT stick
4. CRAFT wooden_pickaxe
5. MINE stone
6. CRAFT stone_pickaxe
7. MINE raw_iron
8. SMELT raw_iron -> iron_ingot
9. CRAFT furnace
10. SMELT cobblestone -> smooth_stone
11. CRAFT blast_furnace
METRICS: {'goal': 'blast_furnace', 'steps': 11, 'max_depth': 7, 'craft': 6, 'mine': 4, 'smelt': 1, 'determinism': 0.8181818181818182, 'unique_items': 11}

=== GOAL: anvil ===
1. MINE oak_log
2. CRAFT oak_planks
3. CRAFT stick
4. CRAFT wooden_pickaxe
5. MINE stone
6. CRAFT stone_pickaxe
7. MINE iron_block
8. MINE raw_iron
9. SMELT raw_iron -> iron_ingot
10. CRAFT anvil
METRICS: {'goal': 'anvil', 'steps': 10, 'max_depth': 6, 'craft': 5, 'mine': 4, 'smelt': 1, 'determinism': 0.8, 'unique_items': 10}

=== GOAL: chest ===
1. MINE oak_log
2. CRAFT oak_planks
3. CRAFT chest
METRICS: {'goal': 'chest', 'steps': 3, 'max_depth': 2, 'craft': 2, 'mine': 1, 'smelt': 0, 'determinism': 0.6666666666666666, 'unique_items': 3}

=== GOAL: barrel ===
1. MINE oak_log
2. CRAFT oak_planks
3. CRAFT cherry_slab
4. CRAFT barrel
METRICS: {'goal': 'barrel', 'steps': 4, 'max_depth': 2, 'craft': 3, 'mine': 1, 'smelt': 0, 'determinism': 0.5, 'unique_items': 4}

=== GOAL: hopper ===
1. MINE oak_log
2. CRAFT oak_planks
3. CRAFT stick
4. CRAFT wooden_pickaxe
5. MINE stone
6. CRAFT stone_pickaxe
7. MINE raw_iron
8. SMELT raw_iron -> iron_ingot
9. CRAFT chest
10. CRAFT hopper
METRICS: {'goal': 'hopper', 'steps': 10, 'max_depth': 7, 'craft': 6, 'mine': 3, 'smelt': 1 'determinism': 0.7, 'unique_items': 10}

=== GOAL: glass ===
1. MINE sand
2. SMELT sand -> glass
METRICS: {'goal': 'glass', 'steps': 2, 'max_depth': 1, 'craft': 0, 'mine': 1, 'smelt': 1, 'determinism': 1.0, 'unique_items': 2}

=== GOAL: stone_bricks ===
1. MINE oak_log
2. CRAFT oak_planks
3. CRAFT stick
4. CRAFT wooden_pickaxe
5. MINE cobblestone
6. SMELT cobblestone -> stone
7. CRAFT stone_bricks
METRICS: {'goal': 'stone_bricks', 'steps': 7, 'max_depth': 5, 'craft': 4, 'mine': 2, 'smelt': 1, 'determinism': 0.7142857142857143, 'unique_items': 7}

=== GOAL: quartz_block ===
1. MINE oak_log
2. CRAFT oak_planks
3. CRAFT stick
4. CRAFT wooden_pickaxe
5. MINE quartz
6. CRAFT quartz_block
METRICS: {'goal': 'quartz_block', 'steps': 6, 'max_depth': 4, 'craft': 4, 'mine': 2, 'smelt': 0, 'determinism': 0.6666666666666666, 'unique_items': 6}

=== GOAL: quartz_bricks ===
1. MINE oak_log
2. CRAFT oak_planks
3. CRAFT stick
4. CRAFT wooden_pickaxe
5. MINE quartz
6. CRAFT quartz_block
7. CRAFT quartz_bricks
METRICS: {'goal': 'quartz_bricks', 'steps': 7, 'max_depth': 5, 'craft': 5, 'mine': 2, 'smelt': 0, 'determinism': 0.7142857142857143, 'unique_items': 7}

=== GOAL: iron_ingot ===
1. MINE oak_log
2. CRAFT oak_planks
3. CRAFT stick
4. CRAFT wooden_pickaxe
5. MINE stone
6. CRAFT stone_pickaxe
7. MINE raw_iron
8. SMELT raw_iron -> iron_ingot
METRICS: {'goal': 'iron_ingot', 'steps': 8, 'max_depth': 6, 'craft': 4, 'mine': 3, 'smelt': 1, 'determinism': 0.75, 'unique_items': 8}

=== GOAL: gold_ingot ===
1. MINE oak_log
2. CRAFT oak_planks
3. CRAFT stick
4. CRAFT wooden_pickaxe
5. MINE stone
6. CRAFT stone_pickaxe
7. MINE raw_iron
8. SMELT raw_iron -> iron_ingot
9. CRAFT iron_pickaxe
10. MINE raw_gold
11. SMELT raw_gold -> gold_ingot
METRICS: {'goal': 'gold_ingot', 'steps': 11, 'max_depth': 9, 'craft': 5, 'mine': 4, 'smelt': 2, 'determinism': 0.8181818181818182, 'unique_items': 11}

=== GOAL: copper_block ===
1. MINE oak_log
2. CRAFT oak_planks
3. CRAFT stick
4. CRAFT wooden_pickaxe
5. MINE stone
6. CRAFT stone_pickaxe
7. MINE copper_block
METRICS: {'goal': 'copper_block', 'steps': 7, 'max_depth': 5, 'craft': 4, 'mine': 3, 'smelt': 0, 'determinism': 0.7142857142857143, 'unique_items': 7}

```

Они более, чем соответствуют реальным планам.

---

## **9. Классификация пользовательских запросов**

### Что было сделано

1. Создано описание каждого GOAL-предмета.
2. Построены эмбеддинги (`multilingual-e5-base`).
3. Реализована функция:

   ```
   запрос → ближайший предмет
   ```

Работает стабильно на английском по следующей схеме:
  - То есть берутся всевозможные целевые действия (они находятся в actions_db);
  - Для каждого предмета создается текстовое описание;
  - Описания кодируются в векторы;
  - Запрос пользователя сравнивается с ними по косинусному сходству.

### Что НЕ получилось

Попытка использовать

- NLLB - Large-scale multilingual model от Meta.Large-scale multilingual model от M*ta.
- M2M100 - модель direct translation между 100+ языками.
- OPUS-MT - статистические Marian-модели перевода.

для мультиязычного перевода дала непредсказуемые результаты, к примеру перевод в ring / curtain / window при запросе кирки.

Причиной является то, что краткие доменные запросы плохо переводятся общими MT-моделями.

Поэтому перевод с других языков отключён полностью (надеюсь, что пока что).

---

## **10. Нормализатор запросов**
Для более точного определения цели был добавлен нормализатор запроса для добавления ключевых слов по типу задачи. Ключевые слова следующие:
```json
{
    "TOOL": "tool instrument pickaxe sword axe weapon кирка меч",
    "BLOCK": "block bricks slab stairs building блок кирпич",
    "MATERIAL": "material ingot gem resource слиток ресурс",
}
```
---
## **11. Полученный результат**

Сформирована полноценная система:

### **Компоненты:**

- генератор действий (`actions_db.json`);
- граф зависимостей;
- детерминированный планировщик;
- классификатор цели (эмбеддинги);
- нормализатор запросов;
- CLI-интерфейс для взаимодействия.

### **Система способна:**

- понять запрос пользователя;
- корректно выбрать предмет;
- построить по нему план;
- гарантировать правильную последовательность действий для STEVE-1.

---
